# 1. Install Dependencies

In [ ]:
# Install required libraries
!pip install -U datasets transformers evaluate optuna peft bitsandbytes
!apt-get install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2. Preprocess Data

In [ ]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)

train_dataset = imdb['train'].shuffle(seed=42)
test_dataset = imdb['test'].shuffle(seed=42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
from datasets import load_dataset
import torch
import optuna
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import evaluate

torch.manual_seed(42)

CHECKPOINT = "microsoft/deberta-v3-base"
save_path = "./deberta-v3"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)




tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# 3. LoRA

In [ ]:
from peft import LoraConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

model

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_proj", "key_proj", "value_proj"],
    modules_to_save=["classifier", "pooler"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

trainable params: 1,034,498 || all params: 185,458,180 || trainable%: 0.5578


In [6]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huangyj2002 (huangyj2002-nanyang-technological-university-singapore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.562600
1000,0.213700
1500,0.192800


TrainOutput(global_step=1564, training_loss=0.31787764446814654, metrics={'train_runtime': 1290.281, 'train_samples_per_second': 38.751, 'train_steps_per_second': 1.212, 'total_flos': 1.33055337426816e+16, 'train_loss': 0.31787764446814654, 'epoch': 2.0})


{'eval_loss': 0.16762007772922516, 'eval_accuracy': 0.93828, 'eval_f1': 0.9388062661114416, 'eval_recall': 0.94688, 'eval_precision': 0.9308690523004326, 'eval_runtime': 263.4456, 'eval_samples_per_second': 94.896, 'eval_steps_per_second': 2.968, 'epoch': 2.0}


In [7]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 31731.82 MB


# 4. Prompt Tuning


In [ ]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 185,031,172 || trainable%: 0.3283


In [ ]:
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

best_lr = 2e-5
best_batch_size = 32
best_epoch = 2

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss
500,0.689600
1000,0.679600
1500,0.676700


TrainOutput(global_step=1564, training_loss=0.6818360477457266, metrics={'train_runtime': 764.229, 'train_samples_per_second': 65.425, 'train_steps_per_second': 2.047, 'total_flos': 1.32376327319424e+16, 'train_loss': 0.6818360477457266, 'epoch': 2.0})


{'eval_loss': 0.6920531392097473, 'eval_accuracy': 0.5018, 'eval_f1': 0.00749063670411985, 'eval_recall': 0.00376, 'eval_precision': 0.9591836734693877, 'eval_runtime': 124.7744, 'eval_samples_per_second': 200.362, 'eval_steps_per_second': 6.267, 'epoch': 2.0}


It appears that the best hyperparameters we obtained from supervised finetuning does not work well with prompt tuning. This is likely due to the randomly initialised prompt embeddings that require higher learning rate to encode useful information about the underlying data distribution. Following the hyperparameter suggestions by the Prompt Tuning paper authored by Lester et al., we will rerun the training with more training steps, higher learning rate and batch size to see if the evaluation accuracy shows any improvement.  

In [ ]:
from peft import PromptTuningConfig, get_peft_model

config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

prompt_config = PromptTuningConfig(
    task_type="SEQ_CLS",
    num_virtual_tokens=20,
    token_dim=768,
    num_transformer_submodules=1,
    num_attention_heads=12,
    num_layers=12,
    tokenizer_name_or_path=CHECKPOINT,
)

prompt_config.modules_to_save = ["classifier", "pooler"]

model = get_peft_model(model, prompt_config)
model.print_trainable_parameters()



pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 607,490 || all params: 185,031,172 || trainable%: 0.3283


In [ ]:
from transformers import EarlyStoppingCallback
from transformers import DataCollatorWithPadding

# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

early_stopping = EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)

batch_size = 32
lr = 1e-3
epoch = 15

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    load_best_model_at_end=True,
    label_names=["labels"],
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

# Reset peak tracking
torch.cuda.reset_peak_memory_stats()

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: huangyj2002 (huangyj2002-nanyang-technological-university-singapore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
500,0.647500,0.402259,0.826880,0.814615,0.760720,0.876729
1000,0.342400,0.155457,0.939760,0.940629,0.954400,0.927250
1500,0.198700,0.145529,0.944120,0.945252,0.964800,0.926481
2000,0.183700,0.149315,0.947480,0.946857,0.935760,0.958221
2500,0.168000,0.130476,0.951400,0.951623,0.956000,0.947285
3000,0.164400,0.130494,0.952720,0.953002,0.958720,0.947352
3500,0.164000,0.124297,0.953120,0.952992,0.950400,0.955598
4000,0.156100,0.141051,0.950680,0.951600,0.969680,0.934181
4500,0.154400,0.135087,0.949120,0.950274,0.972320,0.929205


TrainOutput(global_step=4500, training_loss=0.2421297641330295, metrics={'train_runtime': 6097.885, 'train_samples_per_second': 61.497, 'train_steps_per_second': 1.924, 'total_flos': 3.80914144441536e+16, 'train_loss': 0.2421297641330295, 'epoch': 5.754475703324808})


{'eval_loss': 0.12429697066545486, 'eval_accuracy': 0.95312, 'eval_f1': 0.9529921386170384, 'eval_recall': 0.9504, 'eval_precision': 0.9555984555984556, 'eval_runtime': 273.9757, 'eval_samples_per_second': 91.249, 'eval_steps_per_second': 2.854, 'epoch': 5.754475703324808}


In [ ]:
peak_memory = torch.cuda.max_memory_allocated() / (2 ** 20)
print(f"Peak memory usage: {peak_memory:.2f} MB")

Peak memory usage: 30374.41 MB


# 5. QLoRA

We will not be doing experiments using QLoRA due to existing issue pertaining to mixed precision training for DeBERTaV3 (https://github.com/huggingface/transformers/pull/35336). Instead, we will be demostrating the effect of QLoRA in a separate finetuning experiment comparison for BERT model in a separate notebook.